##### Homework 4 - Rise of the Machines
### Your Name: Alyssa Kern

**COLLABORATED WITH:_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _**

***
***
***

**Submit a PDF export of your notebook (100% PENALTY IF NO PDF IS SUBMITTED)** 

* File > Export Notebook to HTML (you cannot upload .html to Compass), open .html in browser, print to PDF
    - You need to use this option if you have math with `\begin{align} ... \end{align}`.
    - Ensure your code is not cut off

***
***
***

#### Table of Contents


***********************************************************************************************
# Intuition
[TOP](#Homework-4---Rise-of-the-Machines)

**Points are not awarded for correct answers**. 
Instead, points are awarded for explainations of why you are correct or why you are wrong.
You must attempt to answer the question to receive any points.
Points for intuition questions are pass/fail.

Answers such as "I am correct/wrong" or "because the textbook says so" are invalid. 
Show us you know what you are talking about: explain.


## [5 Points] HoML Ch5 Q1

What is the fundamental idea behind Support Vector Machines (classification)?

Answer:
* "widest street"
* "minimizing margin violation"

Why are you correct or wrong?

## [5 Points] HoML Ch5 Q3

Why is it important to scale the inputs when using SVMs?

Answer:

* variables with larger values will dominate the decision threshold/street
* not fit the best (widest) possible street between classes

Why are you correct or wrong?

***********
# Theory
[TOP](#Homework-4---Rise-of-the-Machines)

***********
## [5 Point] Maximial Margin Classifier Part 1
[TOP](#Homework-4---Rise-of-the-Machines)

What kind of kernel does a maximal margin classifier use?

***********
## [5 Point] Maximial Margin Classifier Part 2
[TOP](#Homework-4---Rise-of-the-Machines)

When does a maximal margin classifier fail?

***********
## [10 Point] Multiple Classes
[TOP](#Homework-4---Rise-of-the-Machines)

Using an SVM to predict an 8 class label...

**[5 points]** If we are using one vs. one classification, how many models are required to make a prediction?

**[5 points]** If we are using one vs. rest classification, how many models are required to make a prediction?

****************
# Application
[TOP](#Homework-4---Rise-of-the-Machines)

You will get more independence in this section.
We will be using SVMs for regression (*use the continuous label!*).
Please reference the `sklearn` API reference to determine which functions we need to use: https://scikit-learn.org/stable/modules/classes.html

********
## [9 Points] Preliminaries
[TOP](#Homework-4---Rise-of-the-Machines)

Checklist:

- load all necessary packages and functions
- **[3 points]** deal with variables that should be in the index
- **[2 points]** deal with categorical variables
- **[2 points]** perform a train-test split with a training size of 1/100 and random state of 490
- **[2 points]** standardize the features

In [1]:
import numpy as np
import pandas as pd

# processing
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import plot_confusion_matrix
import statsmodels.api as sm

# alogirthms
from sklearn.svm import SVR, LinearSVR


In [2]:
df = pd.read_csv('hw_data.csv', index_col = ['county', 'occ2010', 'ind1990'])
df.head(1)

,,,id,l_incwage,full_time,degree,l_density,experience,puma_urate,occ_ba,ind_wage,hpi,female,any_kids,black,hispanic
county,occ2010,ind1990,,,,,,,,,,,,,,
1097,3600,831,601,9.862666,0,some college,7.695985,6,5.917822,8.586593,58771.657159,124.54,0,0,0,0


In [3]:
left = df.drop(columns = 'degree')
right = pd.get_dummies(df['degree'], drop_first = True)

df_prepped = pd.concat([left, right], axis = 1)
df_prepped.head(1)

,,,id,l_incwage,full_time,l_density,experience,puma_urate,occ_ba,ind_wage,hpi,female,any_kids,black,hispanic,hs or less,some college
county,occ2010,ind1990,,,,,,,,,,,,,,,
1097,3600,831,601,9.862666,0,7.695985,6,5.917822,8.586593,58771.657159,124.54,0,0,0,0,0,1


In [4]:
y = df_prepped['l_incwage']
x = df_prepped.drop(columns = 'l_incwage')

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 1/100 , random_state = 490)

x_train_std = pd.DataFrame(StandardScaler().fit(x_train).transform(x_train),
                           columns = x_train.columns,
                           index = x_train.index)

x_test_std = pd.DataFrame(StandardScaler().fit(x_test).transform(x_test),
                          columns = x_test.columns, 
                          index = x_test.index)

************
## [4 Points] Null Model
[TOP](#Homework-4---Rise-of-the-Machines)

- **[2 points]** fit the null model
- **[2 points]** save the rmse

In [5]:
#yhat_null = y_train.value_counts().index[0]
#acc_null = np.mean(yhat_null == y_test)
#acc_null


0.0320379838923207

In [ ]:
yhat_null = np.mean(y_train)
rmse_null = np.sqrt(np.mean( (y_test - yhat)**2 ))

******
## [16 Points] Polynomial Kernel
[TOP](#Homework-4---Rise-of-the-Machines)

Using a polynomial kernel, use 5-fold cross-validation grid search to identify optimal values of

- **[4 points]** `degree` - up to a third order
- **[4 points]** `C` - to the closest magnitude ($10^n$)
- **[4 points]** `epsilon` - to the closest magnitude ($10^n$)
- `coef0` - IGNORE

In [6]:
%%time
param_grid = {
    'C': 10.**np.arange(-1, 2, step = 1),
    'degree': [1, 2, 3],
    'epsilon': 10.**np.arange(-1, 1, step = 1),

}

svmr_cv = SVR(kernel = 'poly')

grid_search = GridSearchCV(svmr_cv, param_grid,
                          cv = 5, 
                          scoring = 'neg_root_mean_squared_error').fit(x_train, y_train)

best = grid_search.best_params_
best

CPU times: user 1min 26s, sys: 3.65 s, total: 1min 30s
Wall time: 1min 32s


{'C': 10.0, 'degree': 3, 'epsilon': 1.0}

**[2 points]** Refit on the whole training data using the cross-validated hyperparameters.

In [7]:
# lecture 15
#svmr_poly = SVR(
                     #C = best['C'],
                     #epsilon = best['epsilon'],
                     #degree = best['degree'])
#svmr_poly = svmr_poly.fit(x_train, y_train)

In [ ]:
svmr_poly = SVR(kernel = 'poly',
               degree = best['degree'],
               C = best['C'],
               epsilon = best['epsilon'],)

svmr_poly.fit(x_train, y_train)

**[2 points]** Obtain the RMSE. 

In [8]:
# lecture 15
#acc_poly = svmr_poly.score(x_test, y_test)
#acc_poly
# try this - if it doesnt work, try something else (i dont remember what TA said)

0.0016615228635684653

I used .score instead of .predict. We need to use .predict because we want to predict the labels of the data values on the basis of the trained model. 

In [ ]:
%%time
yhat_poly = svmr_poly.predict(x_test)
rmse_poly = np.sqrt(np.mean( (y_test - yhat_poly)**2 ))

******
## [12 Points] RBF
[TOP](#Homework-4---Rise-of-the-Machines)

Using a radial basis function kernel, use 5-fold cross-validation grid search to identify optimal values of

- **[4 points]** `C` - to the closest magnitude ($10^n$)
- **[4 points]** `gamma` - to the closest magnitude ($10^n$)
- `epsilon` - IGNORE

In [9]:
%%time
param_grid_2 = {
    'C': 10.**np.arange(-1, 4, step = 1),
    'gamma': 10.**np.arange(-7, -3, step = 1)
}

svmr_cv_2 = SVR(kernel = 'rbf')

grid_search_2 = GridSearchCV(svmr_cv_2, param_grid_2,
                          cv = 5, 
                          scoring = 'neg_root_mean_squared_error').fit(x_train, y_train)

best_2 = grid_search_2.best_params_
best_2

CPU times: user 2min 58s, sys: 6.35 s, total: 3min 4s
Wall time: 3min 10s


{'C': 10.0, 'gamma': 1e-06}

**[2 points]** Refit on the whole training data using the cross-validated hyperparameters.

In [10]:
# lecture 15
cvr_rbf = SVR( kernel = 'rbf',
             C = best_2['C'],
             gamma = best_2['gamma'])
cvr_rbf = cvr_rbf.fit(x_train, y_train)

**[2 points]** Obtain the RMSE. 

In [11]:
#acc_rbf = cvr_rbf.score(x_test, y_test)
#acc_rbf

-0.0012407504525924207

In [ ]:
%%time
yhat_poly = svmr_poly.predict(x_test)
rmse_poly = np.sqrt(np.mean( (y_test - yhat_poly)**2))

**********
## [9 Points] Comparison
[TOP](#Homework2---EDA,-Regression,-&-You)


**[2 points]** Print the RMSE for the polynomial and RBF kernel.

In [12]:
print(acc_poly)
print(acc_rbf)

0.0016615228635684653
-0.0012407504525924207


**[3 points]** Caluclate and print the percent improvement from the null model.

In [13]:
print(100*(acc_poly - acc_null)/acc_null)
print(100*(acc_rbf - acc_null)/acc_null)

-94.81389693823174
-103.87274822523966


The values of these RMSE are **VERY** close to the values in HW2, however, we used a magnitude fewer observations with the SVMs!

**[2 points]** Fit a linear regression using `statsmodels` on the training data (all features) used in the support vector machines in this assignment.

In [14]:
# lecture 11 (idk if this is right tho)

#### when it says all features, do we use y_train, x_train, and x_train_std? What about the x_tests?
lin_reg = sm.OLS(y_train, sm.add_constant(x_train)).fit()

#original:
    #sm.OLS(y_train, x_train[['const', 'pct_d_rgdp', 'estabs_exit_rate']]).fit()

#replace x_train with sm.add_constant xtrain 
#want to fit using specifically the statsmodel, want to pass in the xtrain to the sm model
# sm.add_constant adds a colummn of ones and lets you predict the __ as well
# sm.add_constant pass an xtrain

**[2 points]** Print the RMSE from the OLS model.

In [ ]:
# lecture 14
#refit and find the accuracy with the model with the full testing data using the optimal value of `C`
#svc_10 = LinearSVR(C = best_10['C'],
                        #dual = False).fit(x_train_std, y_train)
#acc_10 = svc_10.score(x_test_std, y_test)
#acc_10

########### OR #############
# homework 2
rmse_lin_reg = np.sqrt(np.mean((y_test - lin_reg.predict(x_test))**2))
rmse_lin_reg


**************
## [6 Points] A Number of Observations Tangent
[TOP](#Homework2---EDA,-Regression,-&-You)

**[2 points]** Print the summary.

In [16]:
lin_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              l_incwage   R-squared:                       0.533
Model:                            OLS   Adj. R-squared:                  0.532
Method:                 Least Squares   F-statistic:                     638.2
Date:                Mon, 22 Mar 2021   Prob (F-statistic):               0.00
Time:                        19:51:05   Log-Likelihood:                -9093.4
No. Observations:                7858   AIC:                         1.822e+04
Df Residuals:                    7843   BIC:                         1.832e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            8.6085      0.078    110.136      0.000       8.455       8.762
id             6.28e-11   2.17e-10      0.290      0.772   -3.62e-10    4.87e-10
full_time        1.1444      0.024     48.573      0.000       1.098       1.191
l_density        0.0194      0.009      2.183      0.029       0.002       0.037
experience       0.0170      0.001     24.792      0.000       0.016       0.018
puma_urate      -0.0137      0.005     -2.878      0.004      -0.023      -0.004
occ_ba           0.0069      0.000     17.714      0.000       0.006       0.008
ind_wage      9.654e-06   4.69e-07     20.572      0.000    8.73e-06    1.06e-05
hpi              0.0006      0.000      2.096      0.036    3.84e-05       0.001
female          -0.2704      0.018    -15.174      0.000      -0.305      -0.236
any_kids         0.1845      0.020      9.233      0.000       0.145       0.224
black           -0.1441      0.029     -4.927      0.000      -0.201      -0.087
hispanic        -0.0941      0.036     -2.596      0.009      -0.165      -0.023
hs or less      -0.4805      0.028    -17.467      0.000      -0.534      -0.427
some college    -0.2935      0.024    -12.054      0.000      -0.341      -0.246
==============================================================================
Omnibus:                     1814.772   Durbin-Watson:                   1.996
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             7813.323
Skew:                          -1.071   Prob(JB):                         0.00
Kurtosis:                       7.390   Cond. No.                     7.09e+08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.09e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Compare the significance levels with HW2. 

**[3 points]** All else equal, how does the number of observations influence your statistical significance?

Refer back to the types of error in lecture 8.

**[3 points]** What does an increase in observations increase?

**************
## [12 Points] A Simple Problem
[TOP](#Homework2---EDA,-Regression,-&-You)

- **[4 points]** Resample the data with a 50-50 train-test split using a random state of 490.

In [18]:
y = df_prepped['l_incwage']
x = df_prepped.drop(columns = 'l_incwage')

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 50/100 , random_state = 490)

x_train_std = pd.DataFrame(StandardScaler().fit(x_train).transform(x_train),
                           columns = x_train.columns,
                           index = x_train.index)

x_test_std = pd.DataFrame(StandardScaler().fit(x_test).transform(x_test),
                          columns = x_test.columns, 
                          index = x_test.index)

- **[4 points]** Print the RMSE of a new OLS model fitted on the newly split data.

In [ ]:
# lecture 11; in comparison section above --> substitute x_train for ___.
new_OLS_mod = sm.OLS(y_train, sm.add_constant(x_train)).fit()

rmse_new_OLS_mod = np.sqrt(np.mean((y_test - new_OLS_mod.predict(x_test))**2))
rmse_new_OLS_mod

**[4 points]** Based upon the $\approx$30% improvement and nearly identical RMSE with either 50% or 0.5% sized training data, would you say that predicting wages is a relatively simple or relatively complex prediction problem?

**[for fun]** Print the summary of the second OLS model. 
Verify the p-values are truly smaller with more observations.

In [ ]:
new_OLS_mod.summary()